# COMP 550 Project - Data Cleaning 

In [1]:
# Import block
import os
import pickle
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
nltk.download('punkt') # link to documentation on punkt tokenizers: https://www.nltk.org/_modules/nltk/tokenize/punkt.html

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Samy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data and Storage
### This notebook assumes that you've downloaded the [20 newsgroups dataset](https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups) and unpacked the tarball in the same directory as this notebook. 

### You should therefore have a folder called "20_newsgroup" in the same directory as this notebook. The "20_newsgroup" folder should have 20 subfolders ("comp.os.ms-windows.misc", "soc.religion.christian", "rec.sport.baseball", etc...), each containing a long list of files.

# File Format
### Fortunately, the files are all raw text. Unfortunately, their headers can vary quite a lot. Take the following examples:

##### Sample file #1 from alt.atheism

In [59]:
with open(os.path.abspath(os.path.join(os.getcwd(), "20_newsgroups", "alt.atheism", "51128"))) as atheism_sample_file_1:
    print(''.join(atheism_sample_file_1.readlines()))

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!noc.near.net!news.centerline.com!uunet!olivea!sgigate!sgiblab!adagio.panasonic.com!nntp-server.caltech.edu!keith
From: keith@cco.caltech.edu (Keith Allan Schneider)
Newsgroups: alt.atheism
Subject: Re: Keith Schneider - Stealth Poster?
Message-ID: <1pia4eINNqjg@gap.caltech.edu>
Date: 2 Apr 93 21:10:38 GMT
References: <mam.03kh@mouse.cmhnet.org> <1p6s0cINNhg6@gap.caltech.edu> <1993Mar29.195958.13915@blaze.cs.jhu.edu> <1p8rajINNf3n@gap.caltech.edu> <1993Mar30.184909.20567@blaze.cs.jhu.edu> <1pcrriINNrd6@gap.caltech.edu> <sandvik-310393173357@sandvik-kent.apple.com>
Organization: California Institute of Technology, Pasadena
Lines: 12
NNTP-Posting-Host: punisher.caltech.edu

sandvik@newton.apple.com (Kent Sandvik) writes:

>>To borrow from philosophy, you don't truly understand the color red
>>until you have seen it.
>Not true, even if you have experienced the color red you

##### Sample file #2 from alt.atheism

In [60]:
with open(os.path.abspath(os.path.join(os.getcwd(), "20_newsgroups", "alt.atheism", "49960"))) as atheism_sample_file_2:
    print(''.join(atheism_sample_file_2.readlines()))

Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:49960 alt.atheism.moderated:713 news.answers:7054 alt.answers:126
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!uunet!pipex!ibmpcug!mantis!mathew
From: mathew <mathew@mantis.co.uk>
Newsgroups: alt.atheism,alt.atheism.moderated,news.answers,alt.answers
Subject: Alt.Atheism FAQ: Atheist Resources
Summary: Books, addresses, music -- anything related to atheism
Keywords: FAQ, atheism, books, music, fiction, addresses, contacts
Message-ID: <19930329115719@mantis.co.uk>
Date: Mon, 29 Mar 1993 11:57:19 GMT
Expires: Thu, 29 Apr 1993 11:57:19 GMT
Followup-To: alt.atheism
Distribution: world
Organization: Mantis Consultants, Cambridge. UK.
Approved: news-answers-request@mit.edu
Supersedes: <19930301143317@mantis.co.uk>
Lines: 290

Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December

##### Sample file #1 from alt.autos

In [13]:
with open(os.path.abspath(os.path.join(os.getcwd(), "20_newsgroups", "rec.autos", "101553"))) as auto_sample_file_1:
    print(''.join(auto_sample_file_1.readlines()))

Xref: cantaloupe.srv.cs.cmu.edu rec.autos:101553 rec.autos.tech:53404
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!news.tek.com!tekgen!netops
From: netops@tekgen.bv.tek.com (Randy King)
Newsgroups: rec.autos,rec.autos.tech
Subject: RE: headlights problem
Keywords: headlights
Message-ID: <2407@tekgen.bv.tek.com>
Date: 5 Apr 93 23:23:53 GMT
Article-I.D.: tekgen.2407
Followup-To: rec.autos
Organization: Tektronix Inc., Beaverton, Or.
Lines: 6

THANKS TO ALL OF YOU WHO RESPONDED TO MY POSTING. 

THE PROBLEM WITH MY TRUCK'S HEADLIGHTS LOW BEAM PROBLEM WAS A "LOOSE WIRE
 CONNECTION".  IT WAS NOT THE "FUSE" AS A MINORITY OF YOU SUGGESTED.

THANKS AGAIN.



##### Sample file #2 from alt.autos

In [61]:
with open(os.path.abspath(os.path.join(os.getcwd(), "20_newsgroups", "rec.autos", "103338"))) as auto_sample_file_2:
    print(''.join(auto_sample_file_2.readlines()))

Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!usenet.ins.cwru.edu!magnus.acs.ohio-state.edu!csn!ub!dsinc!netnews.upenn.edu!netnews.cc.lehigh.edu!ns1.cc.lehigh.edu!amh2
From: amh2@ns1.cc.lehigh.edu (ALOIS M. HIMSL)
Newsgroups: rec.autos
Subject: Re: Dumbest automotive concepts of all time
Message-ID: <1993Apr24.003549.126206@ns1.cc.lehigh.edu>
Date: 24 Apr 93 00:35:49 GMT
Organization: Lehigh University
Lines: 15

In article <1993Apr15.155325.6329@jpl-devvax.jpl.nasa.gov>, nancy@hayduke (Nancy
 Feagans) writes:
>Ashtrays and cigarette lighters.  These should be an *option*.
>
>--
>
>Nancy J. Feagans     (818) 306-6423
>Jet Propulsion Lab   nancy@jpl-devvax.Jpl.Nasa.Gov
>"Not a shred of evidence exists in favor of the idea that life is serious."
>

You forget that the cigarette lighter plug is essential for plugging in radar
detectors and lights. The ashtrays are also essential because they are great
places to keep change and tokens.
Al H.



# File Format (cont'd)
### The four examples above each had a "Subject" line in their headers, and it might be useful to use the subsequent words in our analysis. I however believe that we should only do so if we can do it consistently (for every file in every newsgroup), so I checked that every file in the 20 newsgroups had a line starting with "Subject:".

In [62]:
root_dir = '\\\\?\\' + os.path.abspath(os.path.join(os.getcwd(), '20_newsgroups'))
file_count = 0
missing_subject_line_count = 0
subdirectory_list = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
for subdir in subdirectory_list:
    files = [ os.path.join(root_dir, subdir, f) for f in os.listdir(os.path.join(root_dir, subdir)) ]
    for current_file in files:
        with open(current_file,'r') as input_file:
            contents = input_file.readlines()
        found_subject_line = False
        for line in contents:
            if "subject:" in line.lower():
                found_subject_line = True
                break
        if not found_subject_line:
            print(f"{current_file} didn't have a subject line.")
            missing_subject_line_count += 1
        file_count += 1

print(f"examined a total of {file_count} files, {missing_subject_line_count} of which didn't have a subject line.")

examined a total of 19997 files, 0 of which didn't have a subject line.


### OK, so every file has a subject line that we can use!

# Content Extraction
### Some heuristics:
1. The subject line for every file will be considered as valid content.
2. All lines below the "Lines: #" line will be considered as valid content. Some invalid lines may be included (e.g. "NNTP-Posting-Host: punisher.caltech.edu", or "sandvik@newton.apple.com (Kent Sandvik) writes:"), but we can process those lines' contents afterwords with NLTK or another english dictionary API.

### super_dictionary is a dictionary with subdirectory::dict() key::value pairs. The values start off as empty dictionaries, but they are eventually populated with filename::list(valid lines) key::value pairs. 

### In case that's not clear, super_dictionary is supposed to end up looking like a tree like this:

    super_dictionary = {

        'alt.atheism' : {

                            '49960' : [ list of all the valid lines in file 49960 in the 'alt.atheism' subdirectory ],

                            '51060' : [ list of all the valid lines in file 51060 in the 'alt.atheism' subdirectory ],

                            ...

                        },

        'comp.graphics' : {

                              '37261': [ list of all the valid lines in file 37261 in the 'comp.graphics' subdirectory ],

                              '37913': [ list of all the valid lines in file 37913 in the 'comp.graphics' subdirectory ],

                              ...

                          } ,

        ...

    }

In [66]:
super_dictionary = {
    'alt.atheism': dict([]), 
    'comp.graphics': dict([]), 
    'comp.os.ms-windows.misc': dict([]), 
    'comp.sys.ibm.pc.hardware': dict([]), 
    'comp.sys.mac.hardware': dict([]), 
    'comp.windows.x': dict([]), 
    'misc.forsale': dict([]), 
    'rec.autos': dict([]), 
    'rec.motorcycles': dict([]), 
    'rec.sport.baseball': dict([]), 
    'rec.sport.hockey': dict([]), 
    'sci.crypt': dict([]), 
    'sci.electronics': dict([]), 
    'sci.med': dict([]), 
    'sci.space': dict([]), 
    'soc.religion.christian': dict([]), 
    'talk.politics.guns': dict([]), 
    'talk.politics.mideast': dict([]), 
    'talk.politics.misc': dict([]), 
    'talk.religion.misc': dict([])
}

### Populating super_dictionary 

In [71]:
root_dir = '\\\\?\\' + os.path.abspath(os.path.join(os.getcwd(), '20_newsgroups'))
file_count = 0
subdirectory_list = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
for subdir in subdirectory_list:
    files = [ os.path.join(root_dir, subdir, f) for f in os.listdir(os.path.join(root_dir, subdir)) ]
    for current_file in files:
        lines_with_valid_content = []
        with open(current_file,'r') as input_file:
            contents = input_file.readlines()

        for i, line in enumerate(contents):
            if "lines: " in line.lower():
                lines_with_valid_content.append(line.rstrip())
                break
        i += 1   
        while i < len(contents):
            lines_with_valid_content.append(contents[i].rstrip())
            i += 1
        
        subdirectory_dictionary = super_dictionary[subdir]
        subdirectory_dictionary[os.path.basename(current_file)] = lines_with_valid_content
        
        file_count += 1

print(f"examined a total of {file_count} files.")

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc
examined a total of 19997 files.


### Visually validating that the super_dictionary's end-value lists have the correct content

In [73]:
alt_atheism_dictionary = super_dictionary["alt.atheism"]
print('\n'.join(alt_atheism_dictionary["49960"]))

Lines: 290

Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December 1992
Version: 1.0

                              Atheist Resources

                      Addresses of Atheist Organizations

                                     USA

FREEDOM FROM RELIGION FOUNDATION

Darwin fish bumper stickers and assorted other atheist paraphernalia are
available from the Freedom From Religion Foundation in the US.

Write to:  FFRF, P.O. Box 750, Madison, WI 53701.
Telephone: (608) 256-8900

EVOLUTION DESIGNS

Evolution Designs sell the "Darwin fish".  It's a fish symbol, like the ones
Christians stick on their cars, but with feet and the word "Darwin" written
inside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.

Write to:  Evolution Designs, 7119 Laurel Canyon #4, North Hollywood,
           CA 91605.

People in the San Francisco Bay area can get Darwin Fish from Lynn Gold --
try mailing <figmo@netcom.com>.  For net people who go to Lynn d

### Pickling super_dictionary into 20_newsgroup_content_dictionary.pickle in the root directory

In [75]:
with open('20_newsgroup_content_dictionary.pickle', 'wb') as handle:
    pickle.dump(super_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('20_newsgroup_content_dictionary.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(super_dictionary == b)

True


# Data Cleaning
### For every leaf node (file) in the super_dictionary, we need to "clean" the leaf node's list of valid lines. 
### This can be done in a number of ways: here I've used NLTK's punkt tokenizer (https://www.nltk.org/_modules/nltk/tokenize/punkt.html), as it can tokenize with non-alphanumeric characters as well as with spaces. See the toy example below.

In [8]:
toy_example_list_of_valid_lines = ["NNTP-Posting-Host: punisher.caltech.edu",
                                   "sandvik@newton.apple.com (Kent Sandvik) writes:",
                                   ">>To borrow from philosophy, you don't truly understand the color red",
                                   ">>until you have seen it."] # taken from /alt.atheism/51128
#help(re.search)

def has_some_alphanumeric_characters(line):
    if re.search('[a-zA-Z]', line):
        return True
    else:
        return False

def tokenize(list_of_lists):
    tokenized_list = []
    for line in list_of_lists:
        tokens = [word for word in nltk.word_tokenize(line) if has_some_alphanumeric_characters(word)]
        print('The line "{}" becomes:\n{}\n\n'.format(str(line), ', '.join(tokens)))
        for tok in tokens:
            tokenized_list.append(tok)
    return tokenized_list

tokenized_toy_example = tokenize(toy_example_list_of_valid_lines)

print(f"\nThe tokens in the toy example are: {', '.join(tokenized_toy_example)}")

The line "NNTP-Posting-Host: punisher.caltech.edu" becomes:
NNTP-Posting-Host, punisher.caltech.edu


The line "sandvik@newton.apple.com (Kent Sandvik) writes:" becomes:
sandvik, newton.apple.com, Kent, Sandvik, writes


The line ">>To borrow from philosophy, you don't truly understand the color red" becomes:
To, borrow, from, philosophy, you, do, n't, truly, understand, the, color, red


The line ">>until you have seen it." becomes:
until, you, have, seen, it



The tokens in the toy example are: NNTP-Posting-Host, punisher.caltech.edu, sandvik, newton.apple.com, Kent, Sandvik, writes, To, borrow, from, philosophy, you, do, n't, truly, understand, the, color, red, until, you, have, seen, it


### As mentioned above, there are a number of different ways to clean this sort of data. Another (possibly >= useful) method would be to tokenize by spaces and remove (entirely) tokens containing characters which are neither alphanumeric nor punctuation marks (specifically apostrophes and hyphens)

In [3]:
help(Counter)

Help on class Counter in module collections:

class Counter(builtins.dict)
 |  Dict subclass for counting hashable items.  Sometimes called a bag
 |  or multiset.  Elements are stored as dictionary keys and their counts
 |  are stored as dictionary values.
 |  
 |  >>> c = Counter('abcdeabcdabcaba')  # count elements from a string
 |  
 |  >>> c.most_common(3)                # three most common elements
 |  [('a', 5), ('b', 4), ('c', 3)]
 |  >>> sorted(c)                       # list all unique elements
 |  ['a', 'b', 'c', 'd', 'e']
 |  >>> ''.join(sorted(c.elements()))   # list elements with repetitions
 |  'aaaaabbbbcccdde'
 |  >>> sum(c.values())                 # total of all counts
 |  15
 |  
 |  >>> c['a']                          # count of letter 'a'
 |  5
 |  >>> for elem in 'shazam':           # update counts from an iterable
 |  ...     c[elem] += 1                # by adding 1 to each element's count
 |  >>> c['a']                          # now there are seven 'a'
 |  7
 

In [10]:
tokens_and_frequencies = Counter(tokenized_toy_example)
tokens_and_frequencies.most_common()

[('you', 2),
 ('NNTP-Posting-Host', 1),
 ('punisher.caltech.edu', 1),
 ('sandvik', 1),
 ('newton.apple.com', 1),
 ('Kent', 1),
 ('Sandvik', 1),
 ('writes', 1),
 ('To', 1),
 ('borrow', 1),
 ('from', 1),
 ('philosophy', 1),
 ('do', 1),
 ("n't", 1),
 ('truly', 1),
 ('understand', 1),
 ('the', 1),
 ('color', 1),
 ('red', 1),
 ('until', 1),
 ('have', 1),
 ('seen', 1),
 ('it', 1)]